In [ ]:
import time
print(f"Notebook start time: {time.strftime('%I:%M:%S %p, %B %d %Y', time.localtime())}")

In [ ]:
## Install packages if needed
# !pip install selenium
# !pip install chromedriver-autoinstaller
# !pip install Pillow
# !pip install pytesseract

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import random
import gc
from IPython.display import clear_output, Audio, display
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'folder\path\of\your\tesseract.exe'
import requests
from PIL import Image


################# Update with your information ####################
# Set product URL, Target Price, and Login Info
product_url = "https://www.amazon.com/product-of-your-interest"
target_price = 123
email = "email@email.com"
password = "password"
###################################################################

# Set browser driver options  
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
# chrome_options.add_argument("--incognito")
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("window-size=1024,768")

# Check and update chromedriver 
chromedriver_autoinstaller.install()

# Set path for browser driver
PATH = "folder\path\of\your\chromedriver.exe"
# Apply options for browser driver
driver = webdriver.Chrome(PATH, options = chrome_options)

def A_Auto_Buy(): 
    # Sign in to Amazon Account
    signin()
    
    #Go to Amazon product web page
    driver.get(product_url)
    print(driver.title)
    
    # While the product has not been bought
    while True:
        # Run garbage collector
        gc.collect()
        
        # If the Buy Now option is available and the price is less than the threshold 
        if ("Buy Now" in driver.page_source) and (driver.find_element_by_id("priceblock_ourprice").text[1:]) and (float(driver.find_element_by_id("priceblock_ourprice").text[1:].replace(',','')) <= target_price):

            element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.ID , "buy-now-button")) #buy-now-button
                    )
            element.click()

            break
            
        # If refreshed web page requires human verification via Captcha image of letters, use pytesseract to extract characters from image
        elif ("Sorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies." in driver.page_source):    
            image = driver.find_element_by_xpath('/html/body/div/div[1]/div[3]/div/div/form/div[1]/div/div/div[1]/img')
            src = image.get_attribute('src')
            img = Image.open(requests.get(src, stream =True).raw)
            display(img)
            text = tess.image_to_string(img)
            print(text)
            driver.find_element_by_id("captchacharacters").send_keys(text)
        
        # Item not available or price above target. Retry.
        else:
            clear_output()
            print(f"Not in stock or not at the right price point. {time.strftime('%I:%M:%S %p, %B %d %Y', time.localtime())}")
        time.sleep(random.uniform(5,10.5))
        driver.refresh()

    print(f"ORDER PLACED! {time.strftime('%I:%M:%S %p, %B %d %Y', time.localtime())}")
    
def signin():
    # Open browser at Amazon sign in page
    driver.get("https://www.amazon.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2Fgp%2Fcss%2Fhomepage.html%3Ffrom%3Dhz%26ref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&")
    print(driver.title)

    # Enter Amazon Email
    driver.find_element_by_id("ap_email").send_keys(email)
    driver.find_element_by_css_selector("span[class='a-button-inner']").click()
    time.sleep(random.uniform(2,2.5))

    # Enter Amazon Password
    driver.find_element_by_id("ap_password").send_keys(password)
    driver.find_element_by_css_selector("span[class='a-button-inner']").click()
    time.sleep(random.uniform(2,2.5))
    
def allDone():
    display(Audio(url="https://ccrma.stanford.edu/workshops/mir2014/audio/T37-vibraphone-8k.wav", autoplay=True))
    ## Insert whatever audio file you want above

In [ ]:
A_Auto_Buy()
allDone()
time.sleep(3)
driver.quit()